<a href="https://colab.research.google.com/github/andrejarenkow/dashboard_acidentes_poa/blob/main/ASIMOV_Dashboard_acidentes_tr%C3%A2nsito_POA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação de bibliotecas

In [1]:
!pip install jupyter-dash
!pip install dash-bootstrap-components
!pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 7.4 MB/s 
     |████████████████████████████████| 1.6 MB 40.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 3.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importação de bibliotecas

In [2]:
from dash import html, dcc
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc 
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px

pd.set_option('display.max_columns', None)
#from app import app

#Processamento dos dados

[Dicionário de dados](https://dadosabertos.poa.br/dataset/d6cfbe48-ee1f-450f-87f5-9426f6a09328/resource/504c0e16-4cf7-408e-bdea-51ceffd5dac2/download/dd-cat-acidentes.pdf)

In [3]:
df_data = pd.read_csv('https://dadosabertos.poa.br/dataset/d6cfbe48-ee1f-450f-87f5-9426f6a09328/resource/b56f8123-716a-4893-9348-23945f1ea1b9/download/cat_acidentes.csv', sep=';')

#df_data = df_data[['data', 'feridos','feridos_gr',
#                   'mortes', 'morte_post', 'fatais',
#                   'auto', 'taxi','lotacao', 'onibus_urb',
#                   'caminhao', 'moto',
#                   'carroca', 'bicicleta', 'outro', 'cont_vit',
#                   'ups', 'patinete','idacidente', 'longitude',
#                   'latitude', 'tipo_acid',
#                   'dia_sem', 'hora', 'noite_dia', 'regiao']].reset_index(drop=True)

df_data['data'] = pd.to_datetime(df_data['data'], errors='coerce')
df_data.dropna(subset=['data'], inplace=True)
df_data['ano'] = df_data['data'].dt.year
df_data['mes'] = df_data['data'].dt.month
df_data['regiao'] = df_data['regiao'].fillna('-')
limites_lat = [-30.32, -29.95]
limites_lon = [-51.28, -50.97]
#df_data = df_data[(df_data['latitude'] > min(limites_lat)) & (df_data['latitude'] < max(limites_lat))]
#df_data = df_data[(df_data['longitude'] > min(limites_lon)) & (df_data['longitude'] < max(limites_lon))]
df_data = df_data.sort_values('feridos')

df_data

,data_extracao,predial1,queda_arr,data,feridos,feridos_gr,mortes,morte_post,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,longitude,latitude,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao,consorcio,ano,mes
36121,2022-12-01 01:33:44,0.0,0.0,2019-11-16,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,667376,-5.115879e+08,1.288740e+09,AV ELIAS CIRNE LIMA,R PADRE TODESCO,ABALROAMENTO,SÁBADO,07:35:00.0000000,DIA,LESTE,NaN,2019,11
42575,2022-12-01 01:33:44,0.0,0.0,2020-07-10,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,673833,NaN,NaN,R CAIRU,AV BENJAMIN CONSTANT,ABALROAMENTO,SEXTA-FEIRA,14:50:00.0000000,DIA,NORTE,NaN,2020,7
42576,2022-12-01 01:33:44,720.0,0.0,2020-07-13,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,673834,NaN,NaN,R CAIRU,NaN,COLISÃO,SEGUNDA-FEIRA,17:38:00.0000000,DIA,NORTE,NaN,2020,7
42577,2022-12-01 01:33:44,6475.0,0.0,2020-07-11,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,673835,NaN,NaN,AV ASSIS BRASIL,NaN,COLISÃO,SÁBADO,13:40:00.0000000,DIA,NORTE,NaN,2020,7
42578,2022-12-01 01:33:44,262.0,0.0,2020-07-17,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,13,0,673836,NaN,NaN,R BANCO DA PROVINCIA,NaN,ABALROAMENTO,SEXTA-FEIRA,15:30:00.0000000,DIA,SUL,NaN,2020,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11620,2022-12-01 01:33:44,551.0,0.0,2017-11-11,9,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,1,5,0,642803,-5.121337e+01,-3.008241e+01,AV TERESOPOLIS,NaN,COLISÃO,SÁBADO,21:30:00.0000000,NOITE,SUL,NaN,2017,11
67920,2022-12-01 01:33:44,4297.0,0.0,2022-08-15,14,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,5,0,699211,-5.122641e+01,-3.011390e+01,AV CAVALHADA,ESTR JOAO SALOMONI,COLISÃO,SEGUNDA-FEIRA,08:45:00.0000000,DIA,SUL,VIVA SUL (LOTE 4),2022,8
8509,2022-12-01 01:33:44,4255.0,0.0,2017-09-12,21,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,5,0,639645,-5.119355e+01,-3.015848e+01,AV JUCA BATISTA,NaN,ABALROAMENTO,TERÇA-FEIRA,17:00:00.0000000,DIA,SUL,NaN,2017,9
31004,2022-12-01 01:33:44,3711.0,0.0,2019-07-08,25,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,5,0,662255,-8.158712e+08,1.293339e+09,AV BALTAZAR DE OLIVEIRA GARCIA,NaN,COLISÃO,SEGUNDA-FEIRA,14:25:00.0000000,DIA,NORTE,NaN,2019,7


#_controllers.py

https://dash.plotly.com/dash-core-components

In [76]:
from dash import html, dcc
import dash_bootstrap_components as dbc


slider_size=[2017, 2018, 2019, 2020, 2021, 2022]
list_of_locations = {
    'Resetar mapa':'Resetar mapa',
    'Todas':'Todas',
    'Abalroamento':'ABALROAMENTO',
     'Atropelamento':'ATROPELAMENTO',
     'Choque':'CHOQUE',
     'Colisão':'COLISÃO',
     'Queda':'QUEDA',
     'tombamento':'TOMBAMENTO',
     'Eventual':'EVENTUAL',
     'Capotagem':'CAPOTAGEM',
     'Incêndio':'INCÊNDIO',
     'Não cadastrado':'NAO CADASTRADO'
}

tipos_veiculos_envolvidos = ['auto',	'taxi',	'lotacao',	'onibus_urb',	'caminhao',	'moto',	'carroca',	'bicicleta',	'outro',	'patinete']

controllers = dbc.Row([
    #html.Img(id='logo', src=app.get_asset_url('logo_dark.png'), style={'width':'50%'}),
    html.H3('Acidentes de trânsito - POA', style={'margin-top':'30px'}),
    html.P('Elaborado por André Jarenkow, adaptado do projeto da ASIMOV ACADEMY.', style={'font-size':'10px'}),
    html.P('Utilize este dashboard para analisar os acidentes de trânsito em Porto Alegre.'),
    html.H4('Médias por dia', style={'margin-top':'10px', 'text-align': 'center'}),
    dbc.Col(dbc.Card([
        dbc.CardBody([
            html.H5('Acidentes',style={'text-align': 'center'}),
            html.H3(style={'color':'#FFFFFF' , 'text-align': 'center'}, id='media-acidentes-text')
        ])
    ]),md=4),
    dbc.Col(dbc.Card([
        dbc.CardBody([
            html.H5('Feridos', style={'text-align': 'center'}),
            html.H3(style={'color':'#F7F17B', 'text-align': 'center'}, id='media-feridos-text')
        ])
    ]),md=4),
        dbc.Col(dbc.Card([
        dbc.CardBody([
            html.H5('Mortes', style={'text-align': 'center'}),
            html.H3(style={'color':'#DF2935', 'text-align': 'center'}, id='media-obitos-text')
        ])
    ]),md=4),
    html.H4('Tipo de acidente', style={'margin-top':'30px', 'margin-bottom':'20px'}),
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label':i, 'value':j} for i, j in list_of_locations.items()],
        value='Todas',
        placeholder='Selecione uma região'
    ),
    html.H4('Selecione o ano', style={'margin-top':'30px', 'margin-bottom':'20px'}),

    dcc.RangeSlider(
        min=min(slider_size),
        max=max(slider_size),
        marks={i: '{}'.format(i) for i in range(min(slider_size),max(slider_size)+1,1)},
        step=1,
        value=[min(slider_size),max(slider_size)],
        id='slider-ano'
    ),
    html.H4('Selecione os tipos de veículos envolvidos', style={'margin-top':'50px', 'margin-bottom':'50px'}),
    dcc.RadioItems(
        id='check-list-envlovidos',
        options=tipos_veiculos_envolvidos,
        value='auto'
        
    )
])

#_map.py

In [6]:
from dash import dcc
import dash_bootstrap_components as dbc 
import plotly.graph_objects as go

fig = go.Figure()

fig.update_layout(template = 'plotly_dark', paper_bgcolor='rgba(0,0,0,0)')

map = dbc.Row([
    dcc.Loading(id='loading', type='default', children = [dcc.Graph(id='map-graph', figure=fig,style={'height':'80vh'})])    
], style={'height':'80vh'})

#_histogram.py

In [7]:
from dash import dcc
import dash_bootstrap_components as dbc 
import plotly.graph_objects as go

fig = go.Figure()

fig.update_layout(template = 'plotly_dark', paper_bgcolor='rgba(0,0,0,0)')

hist = dbc.Row([
    dcc.Graph(id='hist-graph', figure=fig)
], style={'height':'20vh'})

In [8]:
anos = [2017,2020]
df_data['ano'].isin(range(min(anos),max(anos)))

36121     True
42575    False
42576    False
42577    False
42578    False
         ...  
11620     True
67920    False
8509      True
31004     True
14891     True
Name: ano, Length: 72243, dtype: bool

#Arquivo index.py

Cores - https://plotly.com/python/builtin-colorscales/

In [77]:
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.SLATE])


#=============================
#Layout
app.layout = dbc.Container(
        children=[
                dbc.Row([
                        dbc.Col([controllers], md=3),
                        dbc.Col([map, hist], md=9),
                ])

        ], fluid=True, )

#=============================
#Callbacks
@app.callback([Output('hist-graph', 'figure'),
                Output('map-graph', 'figure'),
               Output('media-acidentes-text', 'children'),
               Output('media-feridos-text', 'children'),
               Output('media-obitos-text', 'children')],
                [Input('location-dropdown', 'value'),
                Input('slider-ano', 'value'),
                 Input('check-list-envlovidos', 'value')])

def update_hist(location, ano, envolvidos):
        if location is None:
                df_intermediate = df_data.copy()
        
        else:
                
                df_intermediate = df_data[df_data['tipo_acid']==location] if location != 'Todas' else df_data.copy()
                df_intermediate = df_intermediate[df_intermediate['ano'].isin(range(min(ano),max(ano)+1))]
                df_intermediate = df_intermediate[df_intermediate[envolvidos]>0]

        hist_fig = px.histogram(df_intermediate, x='data', opacity=0.75,
                                 nbins=(df_intermediate['data'].max() - df_intermediate['data'].min()).days+1, color_discrete_sequence=['indianred'])
        hist_layout = go.Layout(
                margin=go.layout.Margin(l=10, r=0, t=0, b=50),
                showlegend=False,
                template='plotly_dark',
                paper_bgcolor='rgba(0,0,0,0)',
        )
        hist_fig.layout = hist_layout

        

        map_fig = px.scatter_mapbox(df_intermediate, lat='latitude', lon='longitude',
                       center=dict(lat=-30.11201465940859, lon=-51.11035138053364), zoom=10,
                        mapbox_style="carto-darkmatter", color='feridos',size='feridos', opacity=0.5,
                        color_continuous_scale=px.colors.sequential.YlOrRd)
        
        map_fig.update_layout(template='plotly_dark', paper_bgcolor='rgba(0,0,0,0)',
                                margin=go.layout.Margin(l=10, r=10, t=10, b=10))
        
        media_acidentes = '-' if len(df_intermediate) == 0 else round(len(df_intermediate)/len(df_intermediate['data'].unique()),2)

        media_feridos = '-' if len(df_intermediate) == 0 else round((df_intermediate['feridos'].sum())/len(df_intermediate['data'].unique()),2) 

        media_obitos = '-' if len(df_intermediate) == 0 else round((df_intermediate['fatais'].sum())/len(df_intermediate['data'].unique()),2)

        return hist_fig, map_fig, media_acidentes, media_feridos, media_obitos


if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>